In [1]:
import melign
import pandas as pd
dataset = melign.Dataset("dataset_test")
df = pd.read_excel("new_result.xlsx").sort_values(by="Alignment_Precision", ascending=True)
dataset = dataset[df["Sample"][:20]]

In [2]:
df.iloc[:20].describe()

,Unnamed: 0,Alignment_Precision,Alignment_Recall,Alignment_F1,Runtime_seconds
count,20.000000,20.000000,20.000000,20.000000,20.000000
mean,90.450000,0.843783,0.914863,0.875665,49.563669
std,50.672815,0.054081,0.067920,0.043155,25.471426
min,4.000000,0.695187,0.756264,0.780781,21.082742
25%,52.000000,0.821010,0.861771,0.855437,28.997110
50%,94.000000,0.867252,0.932246,0.879770,40.263539
75%,131.250000,0.879728,0.971683,0.910117,68.320290
max,176.000000,0.893617,0.991416,0.939980,104.101515


In [20]:
config = melign.get_default_config(hop_length=8)
config.structural_align = True
config.structural_max_difference = 1
config.melody_min_recurrence = 0.95
config.structural_only = False
config.duration_tolerance = 0.5

In [21]:
result_df, alignments = melign.eval_pipeline(dataset, config=config, save_plot=True, n_jobs=12, verbose=1)

Processing songs (n_jobs=12):   0%|          | 0/20 [00:00<?, ?it/s]

In [22]:
result_df.iloc[-2:]

,Sample,Alignment_Precision,Alignment_Recall,Alignment_F1,Runtime_seconds,Baseline_Precision,Baseline_Recall,Baseline_F1,Precision_Improvement,Recall_Improvement,F1_Improvement
20,Mean,0.941424,0.878560,0.907089,6.871820,0.480162,0.781813,0.589713,0.461262,0.096746,0.317376
21,Median,0.958635,0.907325,0.929251,5.831063,0.541490,0.902409,0.676227,0.417145,0.004915,0.253024


In [19]:
result_df.iloc[-2:]

,Sample,Alignment_Precision,Alignment_Recall,Alignment_F1,Runtime_seconds,Baseline_Precision,Baseline_Recall,Baseline_F1,Precision_Improvement,Recall_Improvement,F1_Improvement
20,Mean,0.932328,0.908983,0.919376,58.399592,0.480162,0.781813,0.589713,0.452166,0.127170,0.329663
21,Median,0.936728,0.927371,0.930698,51.073220,0.541490,0.902409,0.676227,0.395238,0.024962,0.254471


In [8]:
f1_scores = []
precisions = []
recalls = []
f1_old = []
precisions_old = []
recalls_old = []

for i, (name, alignment) in enumerate(alignments.items()):
    percentage = (len(alignment) + sum(m.sum_miss for m in alignment.matches)) / len(dataset[name].melody) * 100
    if percentage > 97.5:
        f1_scores.append(result_df["Alignment_F1"].iloc[i])
        precisions.append(result_df["Alignment_Precision"].iloc[i])
        recalls.append(result_df["Alignment_Recall"].iloc[i])
        f1_old.append(df["Alignment_F1"].iloc[i])
        precisions_old.append(df["Alignment_Precision"].iloc[i])
        recalls_old.append(df["Alignment_Recall"].iloc[i])
from statistics import mean
print(mean(f1_scores), mean(precisions), mean(recalls), len(f1_scores))
print(mean(f1_old), mean(precisions_old), mean(recalls_old), len(f1_old))

0.9285236982683198 0.9470628350031811 0.9126019180474749 12
0.8917030991626395 0.8578697207925089 0.9315359595079746 12


In [45]:
(mean(f1_scores) - mean(f1_old)) / 10, (mean(precisions) - mean(precisions_old)) / 20, (mean(recalls) - mean(recalls_old)) / 10

(np.float64(0.007694481419751664),
 np.float64(0.007693490651589485),
 np.float64(-0.0007715494329569861))

In [46]:
98.13 + 0.769*1.5

99.28349999999999